In [ ]:
# Some points
keras inputs shape 里是不含 batch_size 的
tf 基础框架，张量需要严格对齐，model.fit 则会做一些自定义的方便处理，比如 y_label 是 (None, 1) 或者 (None, )的输入都行
reduction = 'sum' / 'sum_over_batch_size'，sum_over_batch_size 计算出的loss基本是每条样本平均的loss, 等价于 tf.reduce_mean
    
可复现结果：但如果是 BGD，一个batch去算，不应该有这个问题？确认一下BGD
- np.set_random_seed 这些没用，keras_initializer有用

with session() 的sess.run里也会改变变量赋值！要注意原始输入原始数据的隔离
model.compile & sess.run(init) 承担了图及图内 可变tensor 的基本初始化，初始化之前是没有良定义的

============================= 最小子集问题
============================== 几个没解决的 api问题：
1. tf.cast 容易丢失精度? sigmoid_cross_entropy 容易溢出? float32 vs float16, 进一步 int8 优化也可能带来相关风险

2. 同名变量定义了2次，tf图里怎么处理？
3. 可复现结果：
    https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
    https://stackoverflow.com/questions/59075244/if-keras-results-are-not-reproducible-whats-the-best-practice-for-comparing-mo
    目前发现同样的参数，只更新10条样本，新预测结果也会不同。或许 tf2.0 才比较好的解决了这个问题
    train_on_batch 和 fit都无法解决这个问题
    这是SGD算法带来的本质问题么？: https://github.com/keras-team/keras/issues/2743
    

4. fit vs train_on_batch:
    - fit 并不会把weights清零; 这点上说和 train_on_batch 一样
    - batch_size, epochs概念：默认情况下 batch_size = 32, epochs是所有数据被过的轮数
    比如 96条数据 epochs=1, 则实际 iter =3；epochs=3，则实际 iter = 9.
    steps_per_epoch: 可以控制数据并不全部过完，比如每个epoch只过1个iter，那就只过前32条 
    - reset_states: LSTM的问题，https://stackoverflow.com/questions/43882796/when-does-keras-reset-an-lstm-stats
    reset states after batch，train_on_batch 直接做到这一点。model.fit 则保存states, 除非显式指定一个序列已经结束。不确切知道在说啥

=============================== 几个没解决的算法问题
受限的值域范围: 看看影响有多大（mao_model） ---> 故意控制受限值域范围？
已知有影响:同步异步，初始化 (46000 vs 38000)
经过多轮不同起始点的迭代，参数weight是否其实区别很大，但仍然能比较好的刻画结果？

In [ ]:
# Bug

1. from_logits should be True
layers.Dense(activation='sigmoid') + from_logits=False: layers.Dense(activation=None) + from_logits=True. 
默认 from_logits=False
    - 如果 activation = none + from_logits=False, 会影响效果
    
2. TF optimize Bug
SGD vs all GD, batch_size & epoch 是一个平衡, 在iter一定的情况下，也许batch_size越大越好；但epoch一定的情况下，则不一定
一个batch一个batch train很重要；train足够多次也能解决问题

3. keras and tf api of predict result

In [ ]:
# train a classify model to tell if a number is 1.
# save the model.

In [1]:
import os,sys
sys.path.insert(0, "/Users/bytedance/LearningProjects/theory_deep_learning")

In [2]:
PROJECT_PATH= '/Users/bytedance/LearningProjects/theory_deep_learning/generative_model'

In [3]:
import numpy as np
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tensorflow.examples.tutorials.mnist import input_data
from utils.plot_image import plot

In [4]:
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(123)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
python_random.seed(123)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
###  tf.random.set_seed(1234) --->
tf.compat.v1.set_random_seed(1234)

In [5]:
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
tf.keras.backend.set_session(sess)

In [6]:
mnist = input_data.read_data_sets('/Users/bytedance/LearningProjects/MNIST_data', one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/bytedance/LearningProjects/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/bytedance/LearningProjects/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/bytedance/LearningProjects/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/bytedance/LearningProjects/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
X = mnist.train.images
X.shape

# X_new = tf.reshape(X, [-1, 28, 28])
# with tf.Session() as sess:
#     print sess.run(X_new[0:1])

(55000, 784)

In [126]:
y_label = (mnist.train.labels == 1).astype(float)
y_label

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# a simple NN

In [111]:
# Bug: from_logits should be True

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337)),
  tf.keras.layers.Dense(1, activation=None, kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337))
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction='sum'),
              metrics=['accuracy'])

# TF optimize Bug: model.fit(X, y_label, batch_size=50000, epochs=2)
model.fit(X, y_label, epochs=2)
# model.fit(X, y_label_for_tf, epochs=2) # 也是ok的写法

Epoch 1/2
55000/55000 [==============================] - 3s 62us/sample - loss: 0.9841 - acc: 0.9906
Epoch 2/2
55000/55000 [==============================] - 3s 62us/sample - loss: 0.4444 - acc: 0.9958


In [ ]:
pwd = '/Users/bytedance/LearningProjects/theory_deep_learning/generative_model/'
model.save_weights(pwd + 'weights/weights_keras')

In [93]:
model.load_weights(pwd + 'weights/weights_keras')
print model.predict(X[0:10])

for i in range(2):
    model.load_weights(pwd + 'weights/weights_keras')
    print model.train_on_batch(X[0:5], y_label[0:5])
    print '============', model.predict(X[0:10])

for i in range(2):
    model.load_weights(pwd + 'weights/weights_keras')
    print model.fit(X[0:5], y_label[0:5], epochs=1, batch_size=32)

[[-13.515839 ]
 [-11.02475  ]
 [-13.867207 ]
 [-10.416041 ]
 [  6.316296 ]
 [-12.889773 ]
 [  6.7167487]
 [-20.945826 ]
 [ -9.196559 ]
 [-11.598768 ]]
[0.0018535329, 1.0]
============ [[-13.632476]
 [-11.158817]
 [-14.147933]
 [-10.686915]
 [  7.576985]
 [-12.935309]
 [  8.107263]
 [-21.094677]
 [ -9.358425]
 [-11.686738]]
[0.0018535329, 1.0]
============ [[-13.805994 ]
 [-11.368531 ]
 [-14.37539  ]
 [-10.927068 ]
 [  7.9972553]
 [-13.095317 ]
 [  8.5774145]
 [-21.313025 ]
 [ -9.50106  ]
 [-11.866906 ]]
5/5 [==============================] - 0s 316us/sample - loss: 0.0019 - acc: 1.0000


In [14]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(
      128, activation='relu', 
      kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337),
      bias_initializer=tf.keras.initializers.Constant(value=0.1)
  ),
  tf.keras.layers.Dense(
      1, activation=None, 
      kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337),
      bias_initializer=tf.keras.initializers.Constant(value=0.1)
  )
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X, y_label, epochs=1)


55000/55000 [==============================] - 3s 48us/sample - loss: 0.0299 - acc: 0.9904


In [ ]:
model_v2.predict(X)

In [132]:
#==================== INIT 初始化已经得到了很好的控制。但是训练结果仍然有微小的区别
# Bug: keras and tf api of predict result: 把 model_v2.predict 放到 sess.run 模型图里就出问题
for i in range(2):
    model_v2 = tf.keras.models.Sequential([
      tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337)),
      tf.keras.layers.Dense(1, activation=None, kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337))
    ])
    print model_v2.predict(X[0:10])
    
    logits_ = model_v2.predict(X)
    
    
    with tf.Session() as sess:
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits= logits_,
            labels= tf.cast(np.expand_dims(y_label, axis=1), tf.float32)
        ))
        print 'loss', sess.run(loss)
    print '====================='

[[-0.07627121]
 [-0.03569068]
 [ 0.08577306]
 [-0.26084673]
 [-0.05733486]
 [ 0.13431373]
 [ 0.06105527]
 [-0.16407387]
 [-0.01952023]
 [ 0.00732557]]
loss 0.7065882
[[-0.07627121]
 [-0.03569068]
 [ 0.08577306]
 [-0.26084673]
 [-0.05733486]
 [ 0.13431373]
 [ 0.06105527]
 [-0.16407387]
 [-0.01952023]
 [ 0.00732557]]
loss 0.7065882


In [21]:
#====================== 更改模型参数并重新load？

In [27]:
model_v2 = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337)),
  tf.keras.layers.Dense(1, activation=None, kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337))
])

print model_v2.predict(X[0:10])
model_v2.load_weights(pwd + 'weights/weights_keras')
print model_v2.predict(X[0:10])

[[-0.07627121]
 [-0.03569068]
 [ 0.08577306]
 [-0.26084673]
 [-0.05733486]
 [ 0.13431373]
 [ 0.06105527]
 [-0.16407387]
 [-0.01952023]
 [ 0.00732557]]
[[-14.109363 ]
 [ -9.612924 ]
 [-12.866566 ]
 [ -9.374281 ]
 [  5.5253906]
 [-11.393759 ]
 [  6.166926 ]
 [-20.481354 ]
 [ -9.071522 ]
 [-10.516313 ]]


In [ ]:
#======================= see model predict result

In [22]:
model.predict(X[0:10])

array([[-14.109363 ],
       [ -9.612924 ],
       [-12.866566 ],
       [ -9.374281 ],
       [  5.5253906],
       [-11.393759 ],
       [  6.166926 ],
       [-20.481354 ],
       [ -9.071522 ],
       [-10.516313 ]], dtype=float32)

In [14]:
model.predict_classes(X[0:10])

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0]], dtype=int32)

In [15]:
y_label[0:10]

array([0., 0., 0., 0., 1., 0., 1., 0., 0., 0.])

In [134]:
model.losses

[]

In [ ]:
# a simple NN use tf

In [101]:
print X[0:10]
print 'y: ', y_label[0:10]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y:  [0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]


In [110]:
# Bug: TF optimize Bug, batch_size = 50000

with tf.Session() as sess:
    batch_size = 32
    epoch = 1
    sample_total_num = X.shape[0]
    
    inputs = tf.keras.Input(shape=(784,), dtype="float32")
    labels = tf.keras.Input(shape=(), dtype="float32")
    layers = [
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337)),
        tf.keras.layers.Dense(1, activation=None, kernel_initializer=tf.keras.initializers.RandomNormal(seed=1337))
    ]
    x1 = layers[0](inputs)
    logits = layers[1](x1)
    
    # logits = tf.reduce_sum(logits, axis=1)
    # loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
    
    label_for_tf = tf.expand_dims(labels, axis=1)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=label_for_tf))
    
    train_optim = tf.train.AdamOptimizer().minimize(loss)
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    print sess.run(loss, feed_dict={inputs: X, labels: y_label})
    print sess.run(logits, feed_dict={inputs: X[0:10]})
    
    for _ in range(epoch):
        start_idx = 0
        end_idx = start_idx + batch_size
        
        while end_idx < sample_total_num:
            x_batch = X[start_idx: end_idx]
            y_batch = y_label[start_idx: end_idx]
            sess.run(train_optim, feed_dict={inputs: x_batch, labels: y_batch})
            
            start_idx = end_idx
            end_idx = start_idx + batch_size
    
    print 'loss', sess.run(loss, feed_dict={inputs: X, labels: y_label})
    print 'logits', sess.run(logits, feed_dict={inputs: X[0:10]})
    
    pwd = '/Users/bytedance/LearningProjects/theory_deep_learning/generative_model/'
    path = pwd + 'weights_tf/'
    saver = tf.train.Saver(var_list=tf.trainable_variables(), max_to_keep=None)
    saver.save(sess, path)

0.7065882
[[-0.07627121]
 [-0.03569068]
 [ 0.08577306]
 [-0.26084673]
 [-0.05733486]
 [ 0.13431373]
 [ 0.06105527]
 [-0.16407387]
 [-0.01952023]
 [ 0.00732557]]
loss 0.020643558
logits [[-15.1961155]
 [ -9.166121 ]
 [-13.746687 ]
 [-10.323326 ]
 [  7.644575 ]
 [-11.415566 ]
 [  8.236807 ]
 [-20.396305 ]
 [ -9.598089 ]
 [-10.171268 ]]
INFO:tensorflow:/Users/bytedance/LearningProjects/theory_deep_learning/generative_model/weights_tf/ is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
#================================ for test

In [127]:
with tf.Session() as sess:
    y_label_cast = tf.cast(y_label, tf.float32)
    print sess.run(y_label_cast)

[0. 0. 0. ... 0. 0. 0.]


In [145]:
model.fit(X, y_label, epochs=1)

55000/55000 [==============================] - 4s 68us/sample - loss: 0.2345 - acc: 0.99760s - loss: 0.22


In [146]:
model.layers[0].losses

[]